In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

````{margin}
```{spec} Dalitz Distribution
:id: TR-018
:status: WIP
:tags: physics
```
````

# Dalitz Distribution

<!-- cspell:ignore coeff mmikhasenko msigma Remco -->

:::{epigraph}

Mikhail Mikhasenko [@mmikhasenko](https://github.com/mmikhasenko), Remco de Boer [@redeboer](https://github.com/redeboer)

:::

In [ ]:
%pip install -q ampform==0.14.0 qrules==0.9.7 sympy==1.10.1 tensorwaves[pwa]==0.4.5

This report is an attempt to formulate [this report](https://www.overleaf.com/7229968911cjshysdbfjtj) [behind login] on polarization sensitivity in $\Lambda_c \to p\pi K$ with [SymPy](https://docs.sympy.org) and [TensorWaves](https://tensorwaves.rtfd.io).

In [ ]:
from __future__ import annotations

import itertools
import logging
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import qrules
import sympy as sp
from ampform.sympy import (
    PoolSum,
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
    make_commutative,
)
from attrs import define, frozen
from IPython.display import HTML, Math, display
from ipywidgets import HBox, HTMLMath, Tab, VBox, interactive_output
from matplotlib import cm
from matplotlib.colors import LogNorm
from qrules.particle import Particle
from symplot import create_slider
from sympy.core.symbol import Str
from sympy.physics.matrices import msigma
from sympy.physics.quantum.cg import CG
from sympy.physics.quantum.spin import Rotation as Wigner

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)

PDG = qrules.load_pdg()


def display_definitions(definitions: dict[sp.Symbol, sp.Expr]) -> None:
    latex = R"\begin{array}{rcl}" + "\n"
    for symbol, expr in definitions.items():
        symbol = sp.sympify(symbol)
        expr = sp.sympify(expr)
        lhs = sp.latex(symbol)
        rhs = sp.latex(expr)
        latex += Rf"  {lhs} & = & {rhs} \\" + "\n"
    latex += R"\end{array}"
    display(Math(latex))


def display_doit(
    expr: UnevaluatedExpression, deep=False, terms_per_line: int = 10
) -> None:
    latex = sp.multiline_latex(
        lhs=expr,
        rhs=expr.doit(deep=deep),
        terms_per_line=terms_per_line,
        environment="eqnarray",
    )
    display(Math(latex))


# hack for moving Indexed indices below superscript of the base
def _print_Indexed_latex(self, printer, *args):
    base = printer._print(self.base)
    indices = ", ".join(map(printer._print, self.indices))
    return f"{base}_{{{indices}}}"


sp.Indexed._latex = _print_Indexed_latex

## Amplitude model

Naming convention: $\text{Baryon}(\mathbf{0}) \to \text{Baryon}(\mathbf{1}) P(\mathbf{2}) P(\mathbf{3})$
- **Chain 1**: $M^{23} \to P_2 P_3(23)$
- **Chain 2**: $B^{31} \to P_3\text{B}_1(31)$
- **Chain 3**: $B^{12} \to \text{B}_1P_2(12)$

where $B_1$ stands for a baryon with index (1) in the final state. $B^{31}$ and $B^{12}$ are baryon resonances.
The $M$ and $P$ denote the meson state, a general and pseudoscalar, respectively.

In what follows, the 'Chain 0' is the sum of the three chains.

In [ ]:
Λc = PDG["Lambda(c)+"]
p = PDG["p"]
K = PDG["K-"]
π = PDG["pi+"]
decay_products = {
    1: (π, K),
    2: (p, K),
    3: (p, π),
}
siblings = {
    1: p,
    2: π,
    3: K,
}
chain_ids = {
    1: "K^{**}",
    2: R"\Lambda^{**}",
    3: R"\Delta^{**}",
}

Resonance choices and their $LS$-couplings are as follows:

In [ ]:
resonance_names = {
    1: ["K*(892)0"],
    2: ["Lambda(1520)"],
    3: ["Delta(1232)++"],
}
resonances = {
    chain_id: [PDG[name] for name in names]
    for chain_id, names in resonance_names.items()
}

### Aligned amplitude

In [ ]:
A_K = sp.IndexedBase(R"A^K")
A_Λ = sp.IndexedBase(R"A^{\Lambda}")
A_Δ = sp.IndexedBase(R"A^{\Delta}")

half = sp.S.Half

ζ_0_11 = sp.Symbol(R"\zeta^0_{1(1)}", real=True)
ζ_0_21 = sp.Symbol(R"\zeta^0_{2(1)}", real=True)
ζ_0_31 = sp.Symbol(R"\zeta^0_{3(1)}", real=True)
ζ_1_11 = sp.Symbol(R"\zeta^1_{1(1)}", real=True)
ζ_1_21 = sp.Symbol(R"\zeta^1_{2(1)}", real=True)
ζ_1_31 = sp.Symbol(R"\zeta^1_{3(1)}", real=True)

ζ_0_12 = sp.Symbol(R"\zeta^0_{1(2)}", real=True)
ζ_0_23 = sp.Symbol(R"\zeta^0_{2(3)}", real=True)
ζ_0_31 = sp.Symbol(R"\zeta^0_{3(1)}", real=True)
ζ_1_12 = sp.Symbol(R"\zeta^1_{1(2)}", real=True)
ζ_1_23 = sp.Symbol(R"\zeta^1_{2(3)}", real=True)
ζ_1_31 = sp.Symbol(R"\zeta^1_{3(1)}", real=True)


def formulate_aligned_amplitude(λ_Λc, λ_p):
    _ν = sp.Symbol(R"\nu^{\prime}", rational=True)
    _λ = sp.Symbol(R"\lambda^{\prime}", rational=True)
    return PoolSum(
        A_K[_ν, _λ]
        * Wigner.d(half, λ_Λc, _ν, ζ_0_11)
        * Wigner.d(half, _λ, λ_p, ζ_1_11)
        + A_Λ[_ν, _λ]
        * Wigner.d(half, λ_Λc, _ν, ζ_0_21)
        * Wigner.d(half, _λ, λ_p, ζ_1_21)
        + A_Δ[_ν, _λ]
        * Wigner.d(half, λ_Λc, _ν, ζ_0_31)
        * Wigner.d(half, _λ, λ_p, ζ_1_31),
        (_λ, [-half, +half]),
        (_ν, [-half, +half]),
    )


ν = sp.Symbol("nu")
λ = sp.Symbol("lambda")
formulate_aligned_amplitude(λ_Λc=ν, λ_p=λ)

### Decay chain amplitudes

In [ ]:
sp.KroneckerDelta(1, 1)

In [ ]:
(-1) ** (half - half)

Example of a data container for all decay info. More info on this syntax [here](https://www.attrs.org/en/stable/examples.html):

In [ ]:
@define
class Decay:
    j0: int
    j1: int
    j2: int
    j3: int
    j: int
    LS: int
    ls: int
    chain_ind: int

In [ ]:
def formulate_chain_amplitude(chain_id: int, λ0, λ1):
    resonances = resonance_choices[chain_id]
    if chain_id == 1:
        return formulate_23_amplitude(λ0, λ1, resonances)
    if chain_id == 2:
        return formulate_31_amplitude(λ0, λ1, resonances)
    if chain_id == 3:
        return formulate_12_amplitude(λ0, λ1, resonances)
    raise NotImplementedError


H_prod = sp.IndexedBase(R"\mathcal{H}")
H_dec = sp.IndexedBase(R"\mathcal{K}")

θ23 = sp.Symbol("theta23", real=True)
θ31 = sp.Symbol("theta31", real=True)
θ12 = sp.Symbol("theta12", real=True)

σ1, σ2, σ3 = sp.symbols("sigma1:4", nonnegative=True)
m1, m2, m3 = sp.symbols(R"m_p m_pi m_K", nonnegative=True)


R_K = sp.Symbol(R"\mathcal{R}^{K}", real=True)
R_Λ = sp.Symbol(R"\mathcal{R}^{\Lambda}", real=True)
R_Δ = sp.Symbol(R"\mathcal{R}^{\Delta}", real=True)
Rs = {1: R_K, 2: R_Λ, 3: R_Δ}


def formulate_23_amplitude(λ0, λ1, resonances: list[Resonance]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ - λ1)
                * H_prod[stringify(res), τ, λ1]
                * (-1) ** (half - λ1)
                * R_K
                * Wigner.d(sp.Rational(res.particle.spin), τ, 0, θ23)
                * H_dec[stringify(res), 0, 0],
                (τ, create_spin_range(res.particle.spin)),
            )
            for res in resonances
        ]
    )


def formulate_31_amplitude(λ0, λ1, resonances: list[Resonance]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ)
                * H_prod[stringify(res), τ, 0]
                * Wigner.d(sp.Rational(res.particle.spin), τ, -λ1, θ31)
                * R_Λ
                * H_dec[stringify(res), 0, λ1]
                * (-1) ** (half - λ1),
                (τ, create_spin_range(res.particle.spin)),
            )
            for res in resonances
        ]
    )


def formulate_12_amplitude(λ0, λ1, resonances: list[Resonance]):
    τ = sp.Symbol("tau", rational=True)
    return sp.Add(
        *[
            PoolSum(
                sp.KroneckerDelta(λ0, τ)
                * H_prod[stringify(res), τ, 0]
                * Wigner.d(sp.Rational(res.particle.spin), τ, λ1, θ12)
                * R_Δ
                * H_dec[stringify(res), λ1, 0],
                (τ, create_spin_range(res.particle.spin)),
            )
            for res in resonances
        ]
    )


def formulate_dynamics(decay: Resonance, s, m1, m2):
    l_R = sp.Rational(decay.l_R)
    l_Λc = sp.Rational(decay.l_Λc)
    mass = sp.Symbol(f"m_{{{decay.particle.latex}}}")
    width = sp.Symbol(Rf"\Gamma_{{{decay.particle.latex}}}")
    parameter_defaults[mass] = decay.particle.mass
    parameter_defaults[width] = decay.particle.width
    return RelativisticBreitWigner(s, mass, width, m1, m2, l_R, l_Λc, R)


def stringify(particle: Particle | Resonance) -> Str:
    if isinstance(particle, Resonance):
        particle = particle.particle
    return Str(particle.latex)


def create_spin_range(j):
    return arange(-j, +j)


display(
    formulate_chain_amplitude(1, ν, λ),
    formulate_chain_amplitude(2, ν, λ),
    formulate_chain_amplitude(3, ν, λ),
)

### Helicity coupling values

In [ ]:
parameter_defaults = {}

In [ ]:
dec_couplings = {}
for res in resonance_choices[1]:
    i = stringify(res)
    dec_couplings[H_dec[i, 0, 0]] = 1
for res in resonance_choices[2]:
    p = res.particle
    i = stringify(p)
    dec_couplings[H_dec[i, 0, half]] = 1
    dec_couplings[H_dec[i, 0, -half]] = int(-p.parity) * (-1) ** (
        sp.Rational(p.spin) - half
    )
for res in resonance_choices[3]:
    p = res.particle
    i = stringify(p)
    dec_couplings[H_dec[i, half, 0]] = 1
    dec_couplings[H_dec[i, -half, 0]] = int(-p.parity) * (-1) ** (
        sp.Rational(p.spin) - half
    )
parameter_defaults.update(dec_couplings)
display_definitions(dec_couplings)

The production couplings are written in the LS basis:
$$
\mathcal{H}_{\lambda,\lambda'} = \sum_{LS}
    \left\langle j,\lambda ; j',\lambda' | S, \lambda-\lambda'\right\rangle
    \left\langle L,0 ; S,\lambda-\lambda' | J, \lambda-\lambda'\right\rangle \mathcal{H}_{LS}
$$

In [ ]:
def recoupling(λ1, λ2, j, l, s, j1, j2):
    Δλ = λ1 - λ2
    return CG(j1, λ1, j2, -λ2, s, Δλ) * CG(l, sp.S(0), s, Δλ, j, Δλ)

In [ ]:
prod_couplings = {}
LSs = {1: (sp.S(1), half), 2: (sp.S(2), 3 * half), 3: (sp.S(1), 3 * half)}
j3_values = {1: half, 2: 0, 3: 0}
for chain_id in [1, 2, 3]:
    for res in resonance_choices[chain_id]:
        i = stringify(res)
        l, s = LSs[chain_id]
        j3 = j3_values[chain_id]
        for λ1 in create_spin_range(res.particle.spin):
            for λ2 in create_spin_range(j3):
                c = recoupling(
                    sp.Rational(λ1),
                    sp.Rational(λ2),
                    sp.Rational(Λc.spin),
                    l,
                    s,
                    sp.Rational(res.particle.spin),
                    sp.Rational(j3),
                )
                if c.doit() != 0:
                    prod_couplings[
                        H_prod[i, sp.Rational(λ1), sp.Rational(λ2)]
                    ] = c
display_definitions(prod_couplings)
couplings = dict(dec_couplings)
couplings.update(prod_couplings)
parameter_defaults.update(prod_couplings)

### Intensity expression

Incoherent sum of the amplitudes defined by {ref}`report/017:Aligned amplitude`:

In [ ]:
def formulate_intensity(amplitude_builder):
    return PoolSum(
        sp.Abs(amplitude_builder(λ, ν)) ** 2,
        (λ, [-half, +half]),
        (ν, [-half, +half]),
    )


intensity_expressions = {
    0: formulate_intensity(formulate_aligned_amplitude),
    1: formulate_intensity(partial(formulate_chain_amplitude, 1)),
    2: formulate_intensity(partial(formulate_chain_amplitude, 2)),
    3: formulate_intensity(partial(formulate_chain_amplitude, 3)),
}
intensity_expressions[0]

In [ ]:
A = {1: A_K, 2: A_Λ, 3: A_Δ}
amp_definitions = {}
for chain_id in chain_ids:
    for Λc_heli, p_heli in itertools.product([-half, +half], [-half, +half]):
        symbol = A[chain_id][Λc_heli, p_heli]
        expr = formulate_chain_amplitude(chain_id, ν, λ)
        amp_definitions[symbol] = expr.subs({ν: Λc_heli, λ: p_heli})
display_definitions(amp_definitions)

Now we put all equations together and compute the sum over the indices

In [ ]:
expr = (
    formulate_intensity(formulate_aligned_amplitude)
    .doit()
    .xreplace(amp_definitions)
)
up_to_couplings = expr.doit()
final = up_to_couplings.xreplace(couplings).doit()
final_expanded = sp.expand(final)

The relations of the Wigner angles are used to simplify the final expressions.
One expects that 
 - there are no wigner angles for the intensity of any isobar.
 - the $(ij)$-interference term might depend only on the $\zeta_{i(j)}$ angle.

In [ ]:
wigner_angle_identities = {
    ζ_0_21: -2 * sp.pi + ζ_0_23 + ζ_0_31,
    ζ_1_21: ζ_1_23 + ζ_1_31,
    ζ_0_11: 0,
    ζ_1_11: 0,
}
display_definitions(wigner_angle_identities)

In [ ]:
simplified_intensities = {}
Ts = sp.IndexedBase(R"\mathcal{T}")
# diagonal terms
for i in range(1, 4):
    simplified_intensities[Ts[i, i]] = sp.simplify(
        final_expanded.coeff(Rs[i], 2)
    )
# off-diagonal terms
for ij in {12, 23, 31}:
    i, j = divmod(ij, 10)
    coefficient = final_expanded.coeff(Rs[i] * Rs[j], 1)
    simplified_intensities[Ts[i, j]] = sp.simplify(coefficient / sp.S(2)).subs(
        wigner_angle_identities
    )
#

The total intensity is written a bilinear for on dynamic functions $\mathcal{R}$ with coefficients $T_{ij}$ describing the angular dependence.

In [ ]:
intensity_sum = sum(
    Ts[i, j] * Rs[i] * sp.conjugate(Rs[j])
    for i in range(1, 4)
    for j in range(1, 4)
)
intensity_sum = intensity_sum.subs(
    {Ts[2, 1]: Ts[1, 2], Ts[1, 3]: Ts[3, 1], Ts[3, 2]: Ts[2, 3]}
)
display_definitions({sp.Symbol(R"\mathcal{I}_\text{tot}"): intensity_sum})

In [ ]:
display_definitions(simplified_intensities)